In [55]:
import pandas as pd
import numpy as np

In [84]:
test = pd.read_csv('SOTAB CTA/sotab_v2_cta_test_set.csv')

In [ ]:
grouped_by_table = test.groupby(["table_name"])["label"].apply(','.join).reset_index()
grouped_by_table

In [86]:
X = grouped_by_table[["table_name"]].values
y = np.zeros(shape=(len(grouped_by_table["table_name"].tolist()), len(test["label"].unique())))
for index, row in grouped_by_table.iterrows():
    table_labels = row["label"].split(',')
    count = 0
    
    for label in test["label"].unique():
        if label in table_labels:
            y[index][count] = 1
        else:
            y[index][count] = 0
        count += 1

In [5]:
# ! pip install scikit-learn
# ! pip install scikit-multilearn

In [154]:
from collections import Counter
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size = 0.15)
print('Training set length: '+str(len(X_train)) +', Testing set length: '+ str(len(X_test)))

Training set length: 499, Testing set length: 110


In [155]:
p = pd.DataFrame({
    'train': Counter(str(combination) for row in get_combination_wise_output_matrix(y_train, order=1) for combination in row),
    'test' : Counter(str(combination) for row in get_combination_wise_output_matrix(y_test, order=1) for combination in row)
}).T.fillna(0.0)
p

,"(1,)","(44,)","(2,)","(54,)","(27,)","(17,)","(28,)","(81,)","(9,)","(0,)",...,"(68,)","(59,)","(40,)","(39,)","(38,)","(60,)","(62,)","(61,)","(79,)","(63,)"
train,26,18,53,24,19,26,18,10,38,37,...,9,8,4,10,8,17,9,6,8,8
test,6,5,9,4,5,5,3,1,6,6,...,2,2,1,2,3,4,2,1,2,2


In [156]:
sample_testing_tables = [ table[0] for table in X_test ]

In [ ]:
test_set = test[test["table_name"].isin(sample_testing_tables)]
test_set

In [159]:
test_set.to_csv('SOTAB CTA/sample_test.csv', index=False)